In [34]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from collections import Counter

In [35]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [36]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

In [37]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

# Załadowanie plików z danymi

Pliki z danymi znajduję się w folderze `./data/`:

In [38]:
!ls ./data

test.tsv  train.tsv  valid.tsv


In [39]:
train_data = pd.read_csv('./data/train.tsv', sep='\t')

In [40]:
train_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


W danych walidacyjnych i testowych brakuje nagłówków, więc dodaję je z pliku treningowego.

In [41]:
valid_data = pd.read_csv('./data/valid.tsv', sep='\t', header=None, names=train_data.columns)

In [42]:
valid_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,140461,7622,democracy,2
1,140462,7622,and civic action laudable,3
2,140463,7622,civic action laudable,2
3,140464,7622,action laudable,3
4,140465,7623,Griffin & Co. manage to be spectacularly outra...,4


In [43]:
valid_data.tail()

,PhraseId,SentenceId,Phrase,Sentiment
7795,148256,8068,a storm,2
7796,148257,8068,as a fringe feminist conspiracy theorist,1
7797,148258,8068,a fringe feminist conspiracy theorist,2
7798,148259,8068,fringe feminist conspiracy theorist,1
7799,148260,8068,fringe,2


In [44]:
test_data = pd.read_csv('./data/test.tsv', sep='\t', header=None, names=train_data.columns)

In [45]:
test_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,148261,8068,feminist conspiracy theorist,2
1,148262,8068,conspiracy theorist,2
2,148263,8068,theorist,2
3,148264,8068,named Dirty Dick,2
4,148265,8068,Dirty Dick,2


In [46]:
test_data.tail()

,PhraseId,SentenceId,Phrase,Sentiment
7795,156056,8544,Hearst 's,2
7796,156057,8544,forced avuncular chortles,1
7797,156058,8544,avuncular chortles,3
7798,156059,8544,avuncular,2
7799,156060,8544,chortles,2


Sprawdzam czy istnieją komórki z wartością `NaN`:

In [47]:
list(map(lambda x: x.isnull().any().any(), (train_data, valid_data, test_data)))

[False, False, False]

# Wstępna analiza danych

## Forma i wygląd danych

Zdania oznaczone przez `SentenceId` są podzielone na frazy (`Phrase`), z których każda posiada określony `Sentiment`:

In [48]:
train_data.loc[train_data['SentenceId'] == 42]

,PhraseId,SentenceId,Phrase,Sentiment
1134,1135,42,Vincent Gallo is right at home in this French ...,3
1135,1136,42,Vincent Gallo,2
1136,1137,42,Vincent,2
1137,1138,42,Gallo,2
1138,1139,42,is right at home in this French shocker playin...,3
1139,1140,42,is right at home in this French shocker playin...,3
1140,1141,42,is right at home in this French shocker,2
1141,1142,42,right at home in this French shocker,3
1142,1143,42,right at home,3
1143,1144,42,right,3


Powyższa ramka danych uświadamia, że w skład `Phrase` wchodzą zarówno jednowyrazowe napisy oraz napisy składające się z wielu wyrazów. Warto zauważyć, że niektóre `Phrase` składają się tylko z tzw. *stopwords*. Okazuje się, że frazy będące stopwords to nieznaczna część danych, więc w dalszych krokach będę używał metody wykluczania stopwords:

In [49]:
train_data['Phrase'].apply(lambda x: x in stopwords.words('english')).value_counts(normalize=True)

False    0.999179
True     0.000821
Name: Phrase, dtype: float64

Czy we frazach występują znaki interpunkcyjne:

In [50]:
train_data['Phrase'].apply(lambda x: any([punc in x for punc in string.punctuation])).value_counts(normalize=True)

False    0.573704
True     0.426296
Name: Phrase, dtype: float64

### Rozkład zmiennej zależnej według fraz

Sprawdzam jaki jest rozkład zmiennej zależnej (`Sentiment`) według fraz składających się na zdania (`PhraseId`). Przeważa klasa środkowa, czyli neutralna. Rozkład podobny dla wszystkich trzech zestawów danych. Interesujące jest to, że frazy oznaczone jako skrajne(`0` oraz `4`) są średnio najdłuższe:

In [51]:
def get_by_phrase_stats(set_df):
    no_rows = set_df.shape[0]

    aggregation = {
        'Phrase': {
            'count': 'count',
            'freq': lambda x: x.shape[0]/no_rows,
            'avg_number_of_words': lambda x: np.mean([len(phrase.split()) for phrase in x])
        }    
    }

    return set_df.groupby('Sentiment').agg(aggregation)

In [52]:
list(map(lambda x: get_by_phrase_stats(x), (train_data, valid_data, test_data)))

/home/pawel/.local/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


[          Phrase                              
            count      freq avg_number_of_words
 Sentiment                                     
 0           6318  0.045129           12.035296
 1          24081  0.172008            9.076118
 2          71738  0.512418            5.139466
 3          29602  0.211444            8.404229
 4           8260  0.059000           10.662228,
           Phrase                              
            count      freq avg_number_of_words
 Sentiment                                     
 0            328  0.042051           13.435976
 1           1519  0.194744            9.903226
 2           3788  0.485641            5.714361
 3           1696  0.217436            8.825472
 4            469  0.060128           11.285714,
           Phrase                              
            count      freq avg_number_of_words
 Sentiment                                     
 0            417  0.053462           12.177458
 1           1605  0.205769           

# Zdefiniowane zadania

W dalszych krokach będę próbował określić wartość kolumny `Sentiment` dla każdej z `Phrase`.

# Podejście z pełnymi zdaniami - ustalenie benchmarku

W tym podejściu skupiam się jedynie na pełnych zdaniach. Innymi słowy ignoruję podział zdania na frazy i jedynie na podstawie całego zdania próbuję określić wartość `Sentiment`. Dla fraz wchodzących w skład danego zdania przypiszę wartości `Sentiment` uzyskane dla tego zdania. Ze względu na niedoskonałość tego podejścia będę mógł je potraktować jako swego rodzaju benchmark.

W zestawie treningowym i walidacyjnym zostawiam tylko pełne zdania:

In [53]:
train_data_full_sent_only = train_data.drop_duplicates(subset='SentenceId')

In [54]:
train_data_full_sent_only.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1


In [55]:
valid_data_full_sent_only = valid_data.drop_duplicates(subset='SentenceId')

In [56]:
valid_data_full_sent_only.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,140461,7622,democracy,2
4,140465,7623,Griffin & Co. manage to be spectacularly outra...,4
14,140475,7624,Like The English Patient and The Unbearable Li...,3
49,140510,7625,`` Auto Focus '' works as an unusual biopic an...,3
68,140529,7626,"Very amusing , not the usual route in a thrill...",3


Poniższa funkcja sprowadza dane zdanie do mniej złożonej reprezentacji (nadal literowej):

In [57]:
stemmer = WordNetLemmatizer()

def simplify_sentence(sentence):
    # usuwanie znaków specjalnych (wszystkich poza alfanumerycznymi)
    sentence = re.sub(r'\W', ' ', sentence)
    
    # usuwanie wszystkich pojedynczych liter
    sentence = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', sentence)

    # zamiana na małe litery
    sentence = sentence.lower()
    
    # wyciągnięcie pojedynczych słów
    sentence = sentence.split()
    
    # sprowadzenie słów do form podstawowych (lematów)
    # uwaga: sentence to teraz lista
    sentence = [stemmer.lemmatize(word) for word in sentence]
    
    # powrót do str
    return " ".join(sentence)

In [58]:
train_data_full_sent_only['Phrase_cleaned'] = train_data_full_sent_only['Phrase'].apply(lambda x: simplify_sentence(x))

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
valid_data_full_sent_only['Phrase_cleaned'] = valid_data_full_sent_only['Phrase'].apply(lambda x: simplify_sentence(x))

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Zamieniam uproszczone (wyczyszczone zdania) na reprezentację bag of words. 

`max_features`: zwracam uwagę tylko na 1000 najczęściej występujących słów, 

`min_df` to liczba minimalna liczba zdań, w których musi wystąpić słowo, żeby nie zostać zignorowanym, 

`max_df` to maksymalny odsetek wszystkich zdań, w których słowo może wystąpić - służy do zignorowania słów, które występują w prawie każdym zdaniu

`stop_words` to słowa, które występują na tyle często w języku, że nie wnoszą żadnych informacji w zadaniu klasyfikacji

In [60]:
vectorizer = CountVectorizer(max_features=30, min_df=5, max_df=0.75, stop_words=stopwords.words('english'))  

In [61]:
train_data_vectorized = vectorizer.fit_transform(train_data_full_sent_only['Phrase_cleaned'])

In [62]:
valid_data_vectorized = vectorizer.fit_transform(valid_data_full_sent_only['Phrase_cleaned'])

Dotychczasowa reprezentacja wektorowa zdań mimo że uwzględnia fakt bardzo rzadkich słów (`min_df`) oraz słów występujących w prawie każdym zdaniu (`max_df`) może zostać ulepszona podejściem TFIDF. Podejście to nadaje większą wagę słowom bardziej informatywnym, tzn. tym które występują często w małej ilości zdań.

In [63]:
tfidfconverter = TfidfTransformer()  
train_data_tfidf = tfidfconverter.fit_transform(train_data_vectorized).toarray() 

In [64]:
valid_data_tfidf = tfidfconverter.fit_transform(valid_data_vectorized).toarray() 

Używam lasu losowego z 500 drzewami w celu dokonania klasyfikacji:

In [65]:
rf_classifier = RandomForestClassifier(n_estimators=500, random_state=0) 

In [66]:
rf_classifier.fit(train_data_tfidf, train_data_full_sent_only['Sentiment'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

Przygotowuję dane testowe, na podstawie których dokonam predykcji:

In [67]:
test_data_full_sent_only = test_data.drop_duplicates(subset='SentenceId')

In [68]:
test_data_full_sent_only['Phrase_cleaned'] = test_data_full_sent_only['Phrase'].apply(lambda x: simplify_sentence(x))

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [69]:
test_data_vectorize = vectorizer.fit_transform(test_data_full_sent_only['Phrase_cleaned'])

In [70]:
test_data_tfidf = tfidfconverter.fit_transform(test_data_vectorize).toarray()

In [71]:
y_pred = rf_classifier.predict(test_data_tfidf)

`y_pred` muszą zostać skopiowane do odpowiadających `PhraseId`, aby można było obliczyć dokładność klasyfikatora. Z jednej strony mam tyle unikalnych `SentenceId` w `test_data`:

In [72]:
len(y_pred)

475

... z drugiej strony w `test_data` jest tyle unikalnych `PhraseId`:

In [73]:
test_data.shape[0]

7800

Wykonuję złączenie dwóch ramek danych po `SentenceId`:

In [74]:
sent_only_preds = pd.DataFrame(data={"predicted_class": y_pred, "SentenceId": test_data_full_sent_only['SentenceId']}, index=test_data_full_sent_only.index)

In [75]:
sent_only_preds.head()

,predicted_class,SentenceId
0,3,8068
5,1,8069
23,3,8070
61,4,8071
71,1,8072


In [76]:
test_data_w_predicted = pd.merge(test_data, sent_only_preds, how='left', on='SentenceId')
test_data_w_predicted.head()

,PhraseId,SentenceId,Phrase,Sentiment,predicted_class
0,148261,8068,feminist conspiracy theorist,2,3
1,148262,8068,conspiracy theorist,2,3
2,148263,8068,theorist,2,3
3,148264,8068,named Dirty Dick,2,3
4,148265,8068,Dirty Dick,2,3


Biorąc pod uwagę tragiczne wyniki klasyfikatora za benchmark w dalszych rozważaniach będę uznawał częstość występowania klasy dominującej w zadaniach:

In [77]:
print(accuracy_score(test_data_w_predicted['Sentiment'], test_data_w_predicted['predicted_class']))
print("Benchmark dla accuracy będzie wynosił: ", max(get_by_phrase_stats(test_data)['Phrase']['freq']))

0.20717948717948717
Benchmark dla accuracy będzie wynosił:  0.49


/home/pawel/.local/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


# Modelowanie embeddingsów

W dalszej części analizy będę posługiwał się podejściem z uczenia głębokiego. W przypadku modelowania tekstu potrzebne jest utworzenie wektorów zanurzonych. Na początku spróbuję utworzyć je własnoręcznie, później skorzystam z już obliczonych, publicznie dostępnych.

<hr>

# Embeggings - 24.05

## Czyszczenie danych

Poniżej zdefiniowana funkcja jest zbyt agresywna, ponieważ dla niektórych obserwacji w `Phrase` (np. tych zawierających same *stopwords*) będzie zwracała puste napisy. W jej miejsce stosuję wyrażenie regularne, które usuwa wszystkie znaki poza literami łacińskimi (małymi i wielkimi), cyframi oraz odstępami (np. spacje, taby, nowe linie). Mimo, że wykorzystany w dalszej części obiekt `Tokenizer` filtruje po znakach m.in. znakach odstępu (`\s`) usuwając je, to na razie muszę zostawić odstępy, które stanowią granicę wyrazów (tokenów) w zdaniu.

In [78]:
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return " ".join(tokens)

In [79]:
train_data['Phrase'] = train_data['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))

In [80]:
train_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [81]:
test_data['Phrase'] = test_data['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))

In [82]:
test_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,148261,8068,feminist conspiracy theorist,2
1,148262,8068,conspiracy theorist,2
2,148263,8068,theorist,2
3,148264,8068,named Dirty Dick,2
4,148265,8068,Dirty Dick,2


Na wejściu do sieci musi pojawiać się wektor stałej długości. Dodatkowym wymaganiem jest to, żeby słowa w nim zawarte były zakodowane jako liczby.

In [83]:
X_tr = train_data['Phrase']
y_tr = train_data['Sentiment']

Definiuję `Tokenizer`, którego jednym z argumentów jest m.in. to ile słów z wejścia zachowa. Domyślne ustawienia (`num_words`) to `num_words-1`, co oznacza, że słowa o najniższej częstości występowania nie przejdą etapu tokenizacji. Wydaje się to rozsądnym pomysłem, ponieważ słowa najrzadziej występujące to zazwyczaj te występujące tylko raz. Dodatkowo przerabiam wszystkie znaki na wersje pisane małą literą (argument `lower=True`).

In [84]:
tokenizer = Tokenizer()

W tym kroku tworzę słownik, który ma następującą postać `słownik['the']=1`, czyli każde słowo z zestawu treningowego jest kodowane jako liczba.

In [85]:
tokenizer.fit_on_texts(X_tr.values)

In [86]:
X_ts = test_data['Phrase']

Poniżej elementy zdania zamieniane są na liczby:

In [87]:
X_tr_tokenized = tokenizer.texts_to_sequences(X_tr)
X_ts_tokenized = tokenizer.texts_to_sequences(X_ts)

Poniższy przykład pokazuje, że zdania zostały zamienione na listy długości równej liczbie tokenów w pierwotnych (ale oczyszczonych regexem) zdaniach:

In [88]:
X_tr[55], X_tr_tokenized[55]

('to much of a story', [5, 54, 3, 2, 40])

In [89]:
X_ts[22], X_ts_tokenized[22]

('for excitement', [13, 1472])

Należy wyrównać listy do tej samej długości (długości najliczniejszego pod względem tokenów zdania). Argument `padding='post'` wskazuje, że w krótszych niż `max_len` zdaniach zera będą dodane na końcu:

In [90]:
max_len = max([len(x.split()) for x in train_data['Phrase']])
max_len

48

In [91]:
X_tr_tokenized = pad_sequences(X_tr_tokenized, max_len, padding='post')

In [92]:
X_tr_tokenized

array([[   2,  316,    3, ...,    0,    0,    0],
       [   2,  316,    3, ...,    0,    0,    0],
       [   2,  316,    0, ...,    0,    0,    0],
       ...,
       [ 280,  442, 3148, ...,    0,    0,    0],
       [   8,   28,  482, ...,    0,    0,    0],
       [  28,  482,   17, ...,    0,    0,    0]], dtype=int32)

In [93]:
X_ts_tokenized = pad_sequences(X_ts_tokenized, max_len, padding='post')

Długość tokenizowanych zdań w obu zestawach jest równa:

In [94]:
X_tr_tokenized.shape[1] == X_ts_tokenized.shape[1]

True

### Budowa sieci

Każdy z wektorów zanurzonych w sieci będzie miał wymiar 100. Zgodnie z dokumentacją `input_dim` dla w `Embedding()` musi być równy liczbie słów plus 1 (plus 1, aby ująć `0` za pomocą, którego wypełnia się wektory krótsze niż maksymalny wymiar). `input_length` to argument informujący o długości wejścia.

In [95]:
embeddings_dim = 300
no_epochs = 50
input_dim = len(tokenizer.word_index)+1
model = Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=embeddings_dim, input_length=max_len))

Warstwa Long Short Term Memory (`LSTM`) oraz warstawa klasyfikująca `Dense`, w której umieszczam 5 neuronów (jedna dla każdej z przewidywanych klas).

In [96]:
model.add(LSTM(units=128, dropout=0.33, recurrent_dropout=0.33))
# model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2, return_sequences=False))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(units=5, activation='softmax'))

W0524 22:51:58.606062 140284932192064 deprecation.py:506] From /home/pawel/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:4081: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Warto zwrócić uwagę na funkcję straty `'sparse_categorical_crossentropy'`, która powinna być używana w przypadku, gdy klasy są rozłączne. W naszym przypadku oznacza to, że ostatnia warstwa nie zwraca prawdopodobieństw należenia do różnych klas. Wspomniana funkcja jest używana, gdy zmienna zależna nie jest kodowana w postaci one-hot, lecz jako liczby całkowite (np. 1, 2, 3, 4, 5).

In [97]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['mse', 'acc'])

In [98]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 48, 300)           4621800   
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (None, 128)               219648    
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 4,842,093
Trainable params: 4,842,093
Non-trainable params: 0
_________________________________________________________________


W tym miejscu należy zdefiniować dane walidacyjne, które należy przekazać w argumencie `validation_data`. 

In [99]:
X_val = valid_data['Phrase']
y_val = valid_data['Sentiment']

X_val_tokenized = tokenizer.texts_to_sequences(X_val)
X_val_tokenized = pad_sequences(X_val_tokenized, max_len, padding='post')

In [100]:
X_val_tokenized.shape

(7800, 48)

Aby zaradzić wcześniej wspomnianemu problemowi niezbalansowanych danych treningowych będę odpowiednio ważył obserwację z każdej kategorii. Kategoria najbardziej liczbna (`2`) ma teraz najmniejszą wagę

In [101]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_tr),
                                                 y_tr)
class_weights = dict(enumerate(class_weights))
class_weights

{0: 4.431750553972776,
 1: 1.1627341057265064,
 2: 0.3903063927067942,
 3: 0.9458752786973853,
 4: 3.3898062953995156}

Podczas treningu będę także stosował mechanizm early stopping, który automatycznie zatrzyma trening jeżeli wcześniej określone kryterium nie zostanie spełnione. Proces treningu zostanie zatrzymany, jeżeli model w 5 epokach nie wykaże się wzrostem `accuracy` na zestawie walidacyjnym na poziomie co najmniej 0.003. Korzystam z `min_delta`, aby wykluczyć początkowe etapy treningu, gdzie mogą występować znaczące fluktuacje pomiędzy epokami. Innymi słowy ustalone zasady zaczynają obowiązywać dopiero po przekroczeniu `val_acc` równego 0.5:

In [102]:
early_stopping = EarlyStopping(min_delta=0.001, 
                               monitor='val_acc', 
                               patience=5, 
                               mode='max',
                               verbose=1,
                               baseline=0.5)

Stosuję także podejście pozwalające zapisać wersję modelu z treningu, która cechowała najlepszą wskazaną cechą. Tutaj skupię się na funkcji straty dla zestawu walidacyjnego (która powinna być minimalizowana):

In [103]:
mc = ModelCheckpoint('./models/model_val_loss_best.h5', 
                     monitor='val_loss', 
                     mode='min', 
                     save_best_only=True,
                     verbose=1)

In [104]:
callback = [early_stopping, mc]

In [105]:
model.fit(x=X_tr_tokenized, 
          y=y_tr, 
          epochs=no_epochs, 
          verbose=1,
          batch_size=64,
          validation_data=(X_val_tokenized, y_val),
          class_weight=class_weights,
          callbacks=callback)

Train on 139999 samples, validate on 7800 samples
Epoch 1/50
139968/139999 [============================>.] - ETA: 0s - loss: 1.5888 - mse: 4.2842 - acc: 0.2744
Epoch 00001: val_loss improved from inf to 1.59119, saving model to ./models/model_val_loss_best.h5
139999/139999 [==============================] - 248s 2ms/sample - loss: 1.5888 - mse: 4.2843 - acc: 0.2744 - val_loss: 1.5912 - val_mse: 4.2776 - val_acc: 0.2586
Epoch 2/50
139968/139999 [============================>.] - ETA: 0s - loss: 1.5445 - mse: 4.2879 - acc: 0.3633
Epoch 00002: val_loss improved from 1.59119 to 1.43523, saving model to ./models/model_val_loss_best.h5
139999/139999 [==============================] - 249s 2ms/sample - loss: 1.5445 - mse: 4.2877 - acc: 0.3633 - val_loss: 1.4352 - val_mse: 4.2840 - val_acc: 0.4432
Epoch 3/50
139968/139999 [============================>.] - ETA: 0s - loss: 1.2728 - mse: 4.3089 - acc: 0.5058
Epoch 00003: val_loss improved from 1.43523 to 1.18483, saving model to ./models/model_

Do ostatecznego sprawdzenia jakości modelu będą potrzebna zmienna zależna z zestawu testowego:

In [106]:
y_test = test_data['Sentiment']

In [107]:
model.evaluate(x=X_ts_tokenized, y=y_test)

7800/7800 [==============================] - 3s 328us/sample - loss: 1.2992 - mse: 4.1307 - acc: 0.5231


[1.2991876481129574, 4.1307063, 0.52307695]